In [19]:
import mediapipe as mp
import numpy as np
import cv2
import math
import pandas as pd
from apscheduler.schedulers.background import BackgroundScheduler

In [20]:
chosen_left_eye_idxs  = [362, 385, 387, 263, 373, 380]
chosen_right_eye_idxs = [33,  160, 158, 133, 153, 144]
chosen_mouth_idxs = [
    #upper right mouth
    # 82, 
    38, 
    # 72, 37,

    # 81, 41, 73, 39,
    # 80, 42, 74, 40,
    # 183,184, 185, 191,
    
    # 61, 
    76, 
    # 62, 78,

    #upper middle mouth
    # 14,  13, 0, 11,

    #upper left mouth
    # 312, 
    268, 
    # 302, 267,
    
    # 311, 271, 303, 269,
    # 310, 272, 304, 270,
    # 415, 407, 408, 409,
    
    # 291, 
    292,
    #  308, 306,
    
    # 324,325, 307, 375,

    #botom right mouth
    # 146, 77,
    # 88, 89,90, 91,
    # 178, 179, 180, 181,
    
    # 87, 
    86, 
    # 85, 84,

    #botom middle mouth
    # 12, 15, 16, 17,

    #botom left mouth
    # 318, 319, 320, 321,
    # 402, 403, 404, 405,
    
    # 317,
    316, 
    # 315, 314
    ]

face_data = {
    "right_eyes" : [],
    "left_eyes" : [],
    'mouth' : []
    }

In [21]:
facemesh =  mp.solutions.face_mesh
face = facemesh.FaceMesh(
    static_image_mode=False, 
    min_tracking_confidence=0.5, 
    min_detection_confidence=0.5
    )
draw = mp.solutions.drawing_utils

In [22]:
def EARandMAR(right_eye, left_eye, mouth):
    right_eye = (eucledianDistance(right_eye[2], right_eye[5]) + eucledianDistance(right_eye[4], right_eye[3]))/(2*eucledianDistance(right_eye[0], right_eye[1]))
    left_eye = (eucledianDistance(left_eye[4], left_eye[3]) + eucledianDistance(left_eye[5], left_eye[2]))/(2*eucledianDistance(left_eye[1], left_eye[0]))
    mouth = (eucledianDistance(mouth[0], mouth[2]) + eucledianDistance(mouth[3], mouth[5]))/(2*eucledianDistance(mouth[1], mouth[4]))
    return right_eye, left_eye, mouth

def eucledianDistance(point1, point2):
    return math.sqrt((point2.x - point1.x)**2 + (point2.y - point1.y)**2)

def ARMeans(data):
    return sum(data)/len(data)

def EARAnalsis(data):
    global message
    left_eye, right_eye = ARMeans(data['right_eyes']), ARMeans(data['left_eyes'])
    if(right_eye < 0.3 and left_eye < 0.3):
        message = f'driver mengalami gejala mengantuk karena mata tertutup selama 2 detik | Drowsines Value : {str(left_eye)} {str(right_eye)}'
        print(message)
    else:
        message = f'driver tidak memiliki gejala mengantuk melalui pemantauan mata | Drowsines Value : {str(left_eye)} {str(right_eye)}'
        print(message)
    face_data['right_eyes'] = []
    face_data['left_eyes'] = []


def MARAnalisis(data):
    mouth = ARMeans(data['mouth'])
    
    if(
        mouth > 0.5
       ):
        message2 = f'driver mengalami gejala mengantuk karena menguap | Drowsines Value : ' + str(mouth)
        print(message2)
    else:
        message2 = 'driver tidak memiliki gejala mengantuk melalui pemantauan mulut | Drowsines Value : ' + str(mouth)
        print(message2)
    face_data['mouth'] = []

def DrowsinessDetection(data, scheduler):

    # Add the function to be called every 1 minute
    scheduler.add_job(MARAnalisis, 'interval', seconds=10, args=(data,))
    scheduler.add_job(EARAnalsis, 'interval', seconds=2, args=(data,))

    # Start the scheduler
    scheduler.start()
    

In [23]:
cap =  cv2.VideoCapture(1)
scheduler = BackgroundScheduler()
DrowsinessDetection(face_data, scheduler)

while True:
    _, frame = cap.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    imgH, imgW, _ = frame.shape
    # print(imgH, imgW)
    rgb =  cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    op = face.process(rgb)
    if op.multi_face_landmarks:
        for pt in op.multi_face_landmarks:
            # draw.draw_landmarks(frame, i)
            temp = [[],[],[]]
            left = 1
            right = 1
            mouth = 1
            for no,point in enumerate(pt.landmark):
                if no in chosen_left_eye_idxs:
                    # cv2.circle(frame, (int(point.x * imgW), int(point.y * imgH)), 2, (0, color, 0), -1)
                    cv2.putText(
                        frame, text= str(f'p{left}'), org=(int(point.x * imgW), int(point.y * imgH)),
                        fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.3, color=(0,255,0),
                        thickness=1, lineType=cv2.LINE_AA
                    )
                    temp[0].append(point)
                    left += 1
                if no in chosen_right_eye_idxs:
                    # cv2.circle(frame, (int(point.x * imgW), int(point.y * imgH)), 2, (color, 0, 0), -1)
                    cv2.putText(
                        frame, text= str(f'p{right}'), org=(int(point.x * imgW), int(point.y * imgH)),
                        fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.3, color=(0,255,0),
                        thickness=1, lineType=cv2.LINE_AA
                    )
                    temp[1].append(point)
                    right += 1
                if no in chosen_mouth_idxs:
                    # cv2.circle(frame, (int(point.x * imgW), int(point.y * imgH)), 2, (0, 0, color), -1)
                    cv2.putText(
                        frame, text= str(f'p{mouth}'), org=(int(point.x * imgW), int(point.y * imgH)),
                        fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.3, color=(0,255,0),
                        thickness=1, lineType=cv2.LINE_AA
                    )
                    temp[2].append(point)
                    mouth += 1
            right_eye, left_eye, mouth = EARandMAR(temp[0], temp[1], temp[2])
            face_data['right_eyes'].append(right_eye)
            face_data['left_eyes'].append(left_eye)
            face_data['mouth'].append(mouth)

            # cv2.putText(
            #     frame, text= 'message', org=(20, 40),
            #     fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(0,255,0),
            #     thickness=2, lineType=cv2.LINE_AA
            # )

            # cv2.putText(
            #     frame, text= 'message2', org=(20, 80),
            #     fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(0,255,0),
            #     thickness=2, lineType=cv2.LINE_AA
            # )

            
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    cv2.imshow("windows", frame)
    if cv2.waitKey(1) == ord('q'):
        break
    # break

scheduler.shutdown()
cap.release()
cv2.destroyAllWindows()

driver tidak memiliki gejala mengantuk melalui pemantauan mata | Drowsines Value : 0.35984306892738976 0.33438666304927983
driver tidak memiliki gejala mengantuk melalui pemantauan mata | Drowsines Value : 0.40334831416031997 0.373307775383672


error: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'putText'
> Overload resolution failed:
>  - Can't convert object to 'str' for 'text'
>  - Can't convert object to 'str' for 'text'


Job "schedule_ear_analisis (trigger: interval[0:00:02], next run at: 2023-08-24 09:14:57 +07)" raised an exception
Traceback (most recent call last):
  File "c:\ProgramData\Anaconda3\envs\pkm23\lib\site-packages\apscheduler\executors\base.py", line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
  File "C:\Users\whisn\AppData\Local\Temp\ipykernel_13416\3904740155.py", line 67, in schedule_ear_analisis
    EARAnalsis(data)
  File "C:\Users\whisn\AppData\Local\Temp\ipykernel_13416\3904740155.py", line 18, in EARAnalsis
    left_eye, right_eye = ARMeans(data['right_eyes']), ARMeans(data['left_eyes'])
  File "C:\Users\whisn\AppData\Local\Temp\ipykernel_13416\3904740155.py", line 14, in ARMeans
    return sum(data)/len(data)
ZeroDivisionError: division by zero
Job "schedule_ear_analisis (trigger: interval[0:00:02], next run at: 2023-08-24 09:14:59 +07)" raised an exception
Traceback (most recent call last):
  File "c:\ProgramData\Anaconda3\envs\pkm23\lib\site-packages\apsched

driver tidak memiliki gejala mengantuk melalui pemantauan mulut | Drowsines Value : 0.10317132901943392


Job "schedule_ear_analisis (trigger: interval[0:00:02], next run at: 2023-08-24 09:15:03 +07)" raised an exception
Traceback (most recent call last):
  File "c:\ProgramData\Anaconda3\envs\pkm23\lib\site-packages\apscheduler\executors\base.py", line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
  File "C:\Users\whisn\AppData\Local\Temp\ipykernel_13416\3904740155.py", line 67, in schedule_ear_analisis
    EARAnalsis(data)
  File "C:\Users\whisn\AppData\Local\Temp\ipykernel_13416\3904740155.py", line 18, in EARAnalsis
    left_eye, right_eye = ARMeans(data['right_eyes']), ARMeans(data['left_eyes'])
  File "C:\Users\whisn\AppData\Local\Temp\ipykernel_13416\3904740155.py", line 14, in ARMeans
    return sum(data)/len(data)
ZeroDivisionError: division by zero
Job "schedule_ear_analisis (trigger: interval[0:00:02], next run at: 2023-08-24 09:15:05 +07)" raised an exception
Traceback (most recent call last):
  File "c:\ProgramData\Anaconda3\envs\pkm23\lib\site-packages\apsched

In [ ]:
cap.release()
cv2.destroyAllWindows()